<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
! pip install aix360
! pip install sklearn

/bin/sh: pip: command not found


/bin/sh: pip: command not found


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/bank-additional/bank-additional-full.csv',\
                 sep = ';')

#the algorithms below will fail without removing nr.employed
df = df.drop(['duration', 'poutcome'], axis = 1)

In [4]:
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,2,999,0,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,yes


In [5]:
from sklearn.model_selection import train_test_split
X, y = df.loc[:, df.columns != 'y'], df['y'].map(dict(yes=1, no=0))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [6]:
from aix360.algorithms.rbm import FeatureBinarizer
fb = FeatureBinarizer(negations=True, returnOrd=True)
X_train_fb, dfTrainStd = fb.fit_transform(X_train)
X_test_fb, dfTestStd = fb.transform(X_test)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
# Instantiate BRCG with small complexity penalty and large beam search width
from aix360.algorithms.rbm import BooleanRuleCG
br = BooleanRuleCG(lambda0=1e-3, lambda1=1e-3, CNF=True)

# Train, print, and evaluate model
br.fit(X_train_fb, y_train)
from sklearn.metrics import accuracy_score
print('Training accuracy:', accuracy_score(y_train, br.predict(X_train_fb)))
print('Test accuracy:', accuracy_score(y_test, br.predict(X_test_fb)))
print('Predict Y=0 if ANY of the following rules are satisfied, otherwise Y=1:')


Learning CNF rule with complexity parameters lambda0=0.001, lambda1=0.001


/Users/patti/anaconda3/envs/elyra_ga/lib/python3.7/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/Users/patti/anaconda3/envs/elyra_ga/lib/python3.7/site-packages/cvxpy/expressions/expression.py:550: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)
/Users/patti/anaconda3/e

Initial LP solved


ValueError: cannot reindex from a duplicate axis

In [ ]:
print(br.explain()['rules'])

In [ ]:
#use LRR
from aix360.algorithms.rbm import LogisticRuleRegression
from sklearn.metrics import accuracy_score
lrr = LogisticRuleRegression(lambda0=0.005, lambda1=0.001, useOrd=True)

# """Next we consider a LogRR model, which can improve accuracy at the cost of a more complex but still interpretable model. Specifically, 
# LogRR fits a logistic regression model using rule-based features, where column generation is again used to generate promising candidates
# from the space of all possible rules. Here we are also including unbinarized ordinal features (useOrd=True) in addition to rules. 
# Similar to BRCG, the complexity parameters lambda0, lambda1 penalize the number of rules included in the model and the number of conditions in each rule. 
# the The values for lambda0,
# lambda1 below strike a good balance between accuracy and model complexity, based on our published experience with the FICO HELOC dataset [3]."""

# Train, print, and evaluate model
lrr.fit(X_train_fb, y_train, dfTrainStd)
print('Training accuracy:', accuracy_score(y_train, lrr.predict(X_train_fb, dfTrainStd)))
print('Test accuracy:', accuracy_score(y_test, lrr.predict(X_test_fb, dfTestStd)))
print('Probability of Y=1 is predicted as logistic(z) = 1 / (1 + exp(-z))')
print('where z is a linear combination of the following rules/numerical features:')
lrr.explain()